In [2]:
import pandas as pd
import numpy as np

def inspection(data:list,reverse=False,cut_num=3):
    if reverse:
        data = data[::-1]
    df  = data[0].copy()
    for i in data[1].iloc[:,0].values:
        if i in df.iloc[:,0].values:
            continue
        else:
            temp = pd.DataFrame([i],columns=[df.columns[0]])
            df = pd.concat([df,temp],axis=0)
    df = df.sort_values(by=[df.columns[0]]).interpolate()
    df = df.reset_index(drop=True)
    result = df[df.iloc[:,0].values<cut_num]
    return result

In [3]:
data = pd.read_csv('../../data/all_data.csv')
cfd_index = np.arange(0,20,4)
sim_index = np.arange(2,20,4)
cfd,sim = [],[]
for i in range(len(cfd_index)):
    cfd.append(data.iloc[:,cfd_index[i]:cfd_index[i]+2])
    sim.append(data.iloc[:,sim_index[i]:sim_index[i]+2])

result = pd.DataFrame()
for i in range(len(cfd)):
    data = [cfd[i].dropna(),sim[i].dropna()]
    temp1 = inspection(data)
    temp2 = inspection(data,reverse=True)
    result = pd.concat([result,temp1,temp2],axis=1)

# result.to_csv('../../data/inspection.csv',index=False)

In [33]:
# 均方根误差，RMSE
rmse_index = np.arange(1,20,2)
rmse_data = [result.iloc[:,i] for i in rmse_index]
index = np.arange(0,10,2)
for i in index:
    a = np.asarray(rmse_data[i].dropna())
    b = np.asarray(rmse_data[i+1].dropna())
    print(np.sqrt(np.mean((a-b)**2)))

2949.5677637373824
3026.8623657245575
2367.183077633599
2851.79158647859
2603.2351203060352


In [45]:
# 平均百分比误差，MAPE
temp = []
for i in index:
    a = np.asarray(rmse_data[i].dropna())
    b = np.asarray(rmse_data[i+1].dropna())
    temp.append([a,b,abs((a-b)/a)])

In [46]:
test = pd.DataFrame(temp[0]).T